In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from tensorflow import keras

In [ ]:
data = pd.read_csv('dataset.csv')

print(data.head())
print(data.info())
print(data.describe())

In [ ]:
print(data.isnull().sum())

data = data.dropna()
print(data.isnull().sum())

In [ ]:
# График 1: Распределение целевой переменной `popularity`
plt.figure(figsize=(10, 6))
sns.histplot(data['popularity'], bins=30, kde=True)
plt.title('Распределение популярности')
plt.xlabel('Популярность')
plt.ylabel('Частота')
plt.show()

# Вывод: Распределение популярности песен в нашем наборе данных показывает, что большинство песен имеют среднюю популярность

In [ ]:
# График 2: Распределение числовых признаков
numerical_features = data.select_dtypes(include=['number']).columns.tolist()
data[numerical_features].hist(bins=30, figsize=(15, 10), layout=(5, 3))
plt.suptitle('Распределение числовых признаков')
plt.show()

# Вывод: Распределение числовых признаков показывает, что некоторые признаки, такие как `duration_ms`, имеют широкий диапазон значений

In [ ]:
# График 3: Взаимосвязь между числовыми признаками и популярностью
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_features):
    plt.subplot(5, 3, i+1)
    sns.scatterplot(x=data[col], y=data['popularity'])
    plt.title(f'{col} vs Popularity')
plt.tight_layout()
plt.show()

# Вывод: Эти графики показывают, что некоторые числовые признаки, такие как `danceability` и `energy`, имеют явную зависимость с популярностью

In [ ]:
categorical_features = data.select_dtypes(include=['object']).columns.tolist()
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [ ]:
# График 5: Корреляционная матрица
plt.figure(figsize=(12, 10))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляционная матрица')
plt.show()

# Вывод: Корреляционная матрица показывает, что некоторые признаки имеют сильную корреляцию с популярностью, например, `danceability` и `energy`

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X = data.drop(columns=['popularity'])
y = data['popularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Масштабирование числовых признаков
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Построение и обучение простой модели (линейная регрессия)
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Оценка модели
y_pred = linear_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'Линейная регрессия - MAE: {mae}, RMSE: {rmse}, MAPE: {mape}')

In [ ]:
# Важность признаков с помощью permutation importance
result = permutation_importance(linear_model, X_test, y_test, n_repeats=10, random_state=42, scoring='neg_mean_absolute_error')
importances = pd.Series(result.importances_mean, index=X.columns)

# Визуализация важности признаков
importances.sort_values().plot(kind='barh', figsize=(10,6))
plt.title('Важность признаков')
plt.xlabel('Среднее уменьшение MAE')
plt.show()


In [ ]:
# Модель 1: Decision Tree
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)
tree_scores = cross_val_score(tree_model, X, y, cv=5, scoring='neg_mean_absolute_error')
print(f'Decision Tree - MAE: {-tree_scores.mean()}')

In [ ]:
# Модель 2: Gradient Boosting
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)
gb_scores = cross_val_score(gb_model, X, y, cv=5, scoring='neg_mean_absolute_error')
print(f'Gradient Boosting - MAE: {-gb_scores.mean()}')

In [ ]:
# Модель 3: K-Nearest Neighbors
knn_model = KNeighborsRegressor()
knn_model.fit(X_train, y_train)
knn_scores = cross_val_score(knn_model, X, y, cv=5, scoring='neg_mean_absolute_error')
print(f'K-Nearest Neighbors - MAE: {-knn_scores.mean()}')

In [ ]:
# Модель 4: Нейронная сеть
nn_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])
nn_model.compile(optimizer='adam', loss='mean_squared_error')
nn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
nn_scores = -cross_val_score(nn_model, X, y, cv=5, scoring='neg_mean_absolute_error')
print(f'Нейронная сеть - MAE: {nn_scores.mean()}')



In [ ]:
# Итоговый вывод: На основе результатов кросс-валидации, модель Gradient Boosting показала наилучшие результаты.